In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [3]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 979 
Number of columns: 37

Labels: 
C    557
L    422
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [4]:
# Extract features and class
X = df.drop("label", axis=1)
y = df["label"].astype("int")

In [5]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_train

281    0
357    1
502    0
914    1
512    0
      ..
204    0
53     0
294    1
723    1
815    0
Name: label, Length: 783, dtype: int32

In [7]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []

for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=[0, 1])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=[0, 1])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=[0, 1])
    cm = confusion_matrix(y_test, model_results, labels=[0, 1])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))

# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,SVC,"[1.0, 1.0]",1.000000,"[1.0, 1.0]","[1.0, 1.0]","[[113, 0], [0, 83]]"
1,KNN,"[1.0, 1.0]",1.000000,"[1.0, 1.0]","[1.0, 1.0]","[[113, 0], [0, 83]]"
2,RF,"[1.0, 1.0]",1.000000,"[1.0, 1.0]","[1.0, 1.0]","[[113, 0], [0, 83]]"
3,LR,"[0.974, 1.0]",0.984694,"[1.0, 0.964]","[0.987, 0.982]","[[113, 0], [3, 80]]"
4,DTC,"[0.982, 0.988]",0.984694,"[0.991, 0.976]","[0.987, 0.982]","[[112, 1], [2, 81]]"
5,SGDC,"[0.974, 1.0]",0.984694,"[1.0, 0.964]","[0.987, 0.982]","[[113, 0], [3, 80]]"
6,NB,"[0.807, 0.779]",0.795918,"[0.85, 0.723]","[0.828, 0.75]","[[96, 17], [23, 60]]"


In [8]:
with open("./model/all_sklearn.pkl", "wb") as f:
    pickle.dump(models, f)

In [8]:
with open("C:/Users/Alrowad/Exercise-correction/bicpes_model/model/SVC_model.pkl", "wb") as f:
    pickle.dump(models["SVC"], f)

In [10]:
with open("C:/Users/Alrowad/Exercise-correction/bicpes_model/model/KNN_model.pkl", "wb") as f:
    pickle.dump(models["KNN"], f)

In [11]:
with open("C:/Users/Alrowad/Exercise-correction/bicpes_model/model/RF_model.pkl", "wb") as f:
    pickle.dump(models["RF"], f)

In [12]:
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)